In [61]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.dates import DateFormatter
import geopandas as gpd
import math

In [62]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

loadModules(file_names)

(<Pvwattsv8 at 0x7f1b8b002f10>, <Grid at 0x7f1b8b0286f0>)

In [63]:
planeFile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4054901DF3845C/408/Shading/1.csv"
shadingProfilesDF = pd.read_csv(planeFile, header=None)
tilts = shadingProfilesDF.iloc[0][3:363]


def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)

    # matrix = np.zeros((len(angles) + 1, len(azimuths) + 1), dtype=int)
    # matrix[0, 1:] = azimuths
    # matrix[1:, 0] = angles

    # for i, angle in enumerate(angles):
    #     for j, tilt in enumerate(tilts):
    #         if angle <= tilt:
    #             matrix[i + 1, j + 1] = 100

    return matrix

shadingMatrix = get_matrix(tilts)

In [64]:
planeIdGDF = gpd.read_file("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4054901DF3845C/408/Plane Identification/408.gpkg")
plane = planeIdGDF[planeIdGDF.cluster == 1]

def getInfoRoof(plane):
    tilt = plane.tilt.values[0]
    azimuth = plane.azimuth.values[0]
    area = plane.area.values[0]/math.cos(tilt*math.pi/180)
    return  area, tilt, azimuth
    
area, tilt, azimuth = getInfoRoof(plane)
plane

,cluster,A,B,D,tilt,azimuth,silhouette,geometry
2,1,-0.237036,0.244355,-1.017541e+06,18.8,135.871064,0.850265,"POLYGON ((433933.943 4585214.663, 433933.943 4..."


In [65]:
def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=0.400/(1.879*1.045)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    # return generation_df/area
    return pv.export()["Outputs"]

tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"
generation = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")

In [66]:
generation.keys()

dict_keys(['ac', 'ac_annual', 'ac_annual_pre_adjust', 'ac_monthly', 'ac_pre_adjust', 'alb', 'annual_energy', 'annual_energy_distribution_time', 'aoi', 'capacity_factor', 'capacity_factor_ac', 'city', 'dc', 'dc_monthly', 'dcsnowderate', 'df', 'dn', 'elev', 'gen', 'gh', 'inv_eff_output', 'inverter_efficiency', 'kwh_per_kw', 'lat', 'location', 'lon', 'monthly_energy', 'percent_complete', 'poa', 'poa_monthly', 'shad_beam_factor', 'snow', 'soiling_f', 'solrad_annual', 'solrad_monthly', 'ss_beam_factor', 'ss_gnd_diffuse_factor', 'ss_sky_diffuse_factor', 'state', 'sunup', 'tamb', 'tcell', 'tpoa', 'ts_shift_hours', 'tz', 'wspd'])

In [70]:
sum(generation["poa"])

1729310.0062196068

In [60]:
sum(generation["poa_monthly"])

1729.3100062196077

In [58]:
sum(generation["solrad_monthly"])/12

4.731790645496262

In [102]:
sum(generation["dc_monthly"])

39607.09450683009

In [87]:
days = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
from itertools import accumulate
cumulative_sum = list(accumulate(days))
cumulative_sum

[0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]

In [99]:
DCOut = np.array(generation["dc"]).reshape(365, 24)
averages = np.zeros((12,24))
for month in range(12):
    for hour in range(24):
        averages[month,hour] = np.average(DCOut[cumulative_sum[month]:cumulative_sum[month+1],hour])
averages = averages.reshape(12*24)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.78394081e+01,
       1.93906492e+03, 4.82156395e+03, 9.34980015e+03, 1.06828922e+04,
       1.05325578e+04, 9.40722935e+03, 6.50660864e+03, 2.51250333e+03,
       3.59878346e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.69040672e+03,
       5.18679054e+03, 8.22156695e+03, 1.29343213e+04, 1.40401744e+04,
       1.32676365e+04, 1.13635526e+04, 8.08107566e+03, 4.44322937e+03,
       1.09905491e+03, 8.04986908e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.19173430e+02, 3.14859153e+03,
      

# As a function

In [12]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import os
import geopandas as gpd
import math

In [13]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)
    return matrix


def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=0.400/(1.879*1.045)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    return generation_df/area



In [14]:
file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"

basePath = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/"
neighborhood = "Test_70_el Besòs i el Maresme"
parcelsFolder = basePath + "/Results/" + neighborhood + "/Parcels/"

annual_ac = 0

for parcel in os.listdir(parcelsFolder):
    parcelSubfolder = parcelsFolder + parcel + "/"
    for construction in [x for x in os.listdir(parcelSubfolder) if os.path.isdir(parcelSubfolder + x)]:
        constructionFolder = parcelSubfolder + construction + "/"
        planesGDF = gpd.read_file(constructionFolder + "Plane Identification/" + construction + ".gpkg")
        for cluster in planesGDF.cluster.values:
            shadingFile = constructionFolder + "/Shading/" + str(cluster) + ".csv"
            plane = planesGDF[planesGDF.cluster == cluster]
            if os.path.isfile(shadingFile):
                if(os.stat(shadingFile).st_size > 0):
                    shadingProfilesDF = pd.read_csv(shadingFile, header=None)
                    tilts = shadingProfilesDF.iloc[0][3:363]

                    generation_df = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")
                    annual_ac = generation_df.sum().sum()        
print(annual_ac)

272938.68342764745


In [36]:
shadingFile

'/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3//Results/Test_70_el Besòs i el Maresme/Parcels/4151302DF3845A/139//Shading/12.csv'